In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
from utils.paths import DATA_RAW_DIR

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

import ipywidgets as widgets
from IPython.display import display

In [3]:
path_partidos = str(DATA_RAW_DIR / 'partidos_cebollitas.csv')

print('Path:', path_partidos)
print('Does path exist?', Path(path_partidos).exists())

path_jugadores = str(DATA_RAW_DIR / 'jugadores_cebollitas.csv')

print('Path:', path_jugadores)
print('Does path exist?', Path(path_jugadores).exists())

Path: /Users/jssdev/Dev/Learning/Platzi/platzi-machine-learning/data/raw/partidos_cebollitas.csv
Does path exist? True
Path: /Users/jssdev/Dev/Learning/Platzi/platzi-machine-learning/data/raw/jugadores_cebollitas.csv
Does path exist? True


In [4]:
df_partidos = pd.read_csv(path_partidos, sep=',', encoding='utf-8')
df_jugadores = pd.read_csv(path_jugadores, sep=',', encoding='utf-8')

In [5]:
df_partidos['diferencia_goles'] = df_partidos['goles_local'] - df_partidos['goles_visitante']

In [6]:
X_partidos = df_partidos[['posesion_local (%)', 'tiros_arco_local']]
y_partidos = df_partidos['diferencia_goles']

In [7]:
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_partidos, y_partidos, test_size=0.2, random_state=42)

In [8]:
X_jugadores = df_jugadores[['goles', 'asistencias', 'pases_completados (%)', 'tiros_al_arco']]

In [9]:
pipeline_supervisado = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge(alpha=1.0))
])
pipeline_supervisado.fit(X_train_p, y_train_p)

Pipeline(steps=[('scaler', StandardScaler()), ('ridge', Ridge())])

In [10]:
pipeline_no_supervisado = Pipeline([
    ('scaler', StandardScaler()),
    ('kmeans', KMeans(n_clusters=3, random_state=42))
])
df_jugadores['cluster'] = pipeline_no_supervisado.fit_predict(X_jugadores)

In [ ]:
nuevo_partido = pd.DataFrame({'posesion_local (%)':[58],'tiros_arco_local':[9]})
prediccion_partido = pipeline_supervisado.predict(nuevo_partido)

perfiles_jugadores = df_jugadores[['nombre', 'cluster']].head(10)

# prediccion_partido is a numpy array, so .round(2) is valid
print(f"Predicción resultado partido (dif. goles): {prediccion_partido.round(2)}")
print("\nPerfiles jugadores (primeros 10):")
print(perfiles_jugadores)

Predicción resultado partido (dif. goles): [0.25]

Perfiles jugadores (primeros 10):
                nombre  cluster
0         Tara Alvarez        1
1        Carol Mcclain        2
2        Robert Martin        1
3    Mr. Robert Turner        2
4  Christopher Kennedy        1
5  Dr. Justin Anderson        0
6      Sandra Shepherd        1
7          Mark Bowers        0
8         Kenneth Cook        1
9    Christopher Parks        2


In [12]:
nuevo_partido

,posesion_local (%),tiros_arco_local
0,58,9


In [13]:
prediccion_partido

array([0.25419475])

In [14]:
posesion_widget = widgets.IntSlider(min=40,max=70,description='Posesión %:')
tiros_widget = widgets.IntSlider(min=1,max=15,description='Tiros al arco:')

def predecir_resultados(posesion, tiros):
    datos = pd.DataFrame({'posesion_local (%)':[posesion],'tiros_arco_local':[tiros]})
    pred = pipeline_supervisado.predict(datos)[0]
    print(f"Diferencia de goles predicha: {pred:.2f}")

widgets.interactive(predecir_resultados, posesion=posesion_widget, tiros=tiros_widget)

interactive(children=(IntSlider(value=40, description='Posesión %:', max=70, min=40), IntSlider(value=1, descr…

In [15]:
nuevo_partido = pd.DataFrame({'posesion_local (%)':[57],'tiros_arco_local':[13]})
prediccion_partido = pipeline_supervisado.predict(nuevo_partido)

perfiles_jugadores = df_jugadores[['nombre', 'cluster']].head(10)

# prediccion_partido is a numpy array, so .round(2) is valid
print(f"Predicción resultado partido (dif. goles): {prediccion_partido.round(2)}")
print("\nPerfiles jugadores (primeros 10):")
print(perfiles_jugadores)

Predicción resultado partido (dif. goles): [-0.]

Perfiles jugadores (primeros 10):
                nombre  cluster
0         Tara Alvarez        1
1        Carol Mcclain        2
2        Robert Martin        1
3    Mr. Robert Turner        2
4  Christopher Kennedy        1
5  Dr. Justin Anderson        0
6      Sandra Shepherd        1
7          Mark Bowers        0
8         Kenneth Cook        1
9    Christopher Parks        2
